In [2]:
using DataFrames, CmdStan, RecursiveArrayTools, JLD, LinearAlgebra, cmdstan_utils, Plots, Distributions, Statistics, BlockArrays
import Pandas.read_csv
plotlyjs()

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-1003336297105055619\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-8736013941005100682\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-1003336297105055619\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-562314776602369467\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-13094782589685000882\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-562314776602369467\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

Plots.PlotlyJSBackend()

┌ Warning: /Users/tomroschinger/.julia/packages/Plots/FYxdA/src/Plots.jl/ is not an existing directory, Revise is not watching
└ @ Revise /Users/tomroschinger/.julia/packages/Revise/jVsKo/src/Revise.jl:492


First we load in a datafile. Since Julia DataFrames has some issues with importing this file (or more like I have problems with it), I chose to use Pandas and then transform it into Julia DataFrames.

In [22]:
df = read_csv("../data/RegSeq/sequence_counts/ykgEarcAdataset_alldone_with_large", delim_whitespace=true)
df = DataFrames.DataFrame(df)
first(df, 5)

,ct,ct_0,ct_1,seq
,Float64,Float64,Float64,String
1,4.0,4.0,0.0,ACAATTTCACCATAAAATGTCGGCGTTGCCGAAAGAAATAAAATGAGGTATTGCATTTGACGTTTGGATGAAAGATTTTCATTTGTCCTACAATTGCGGGGTGGTATGTGGCTAGCCCATTAAAAAAGAACGCCATATTTATTGATGATTGACACCGCGGCCCAGCCAATCTATACGCCT
2,19.0,13.0,6.0,ACGAGTTCCCCATAAAATTTGAGCGATGCCGAAAGAAATAAAAGTAGTTATTACATTCGTCGTATGGATGAAGGCTTATGATTTGTCATACAGAGGAGGGGTGGTATCTTGATGGCCAATGAAATCAGAACGCCATATTTATTGATGATTGATCCCCCGGATTTTAGTGTAAGAACGGCT
3,5.0,5.0,0.0,ACGAGTTCCCCATAAAATTTGAGCGATGCCGAAAGAAATAAAAGTAGTTATTACATTCGTCGTATGGATGAAGGCTTATGATTTGTCATACAGAGGAGGGGTGGTATCTTGATGGCCAATGAAATCAGAACGCCATATTTATTGATGATTGATCCCCCGGTCGTCTTCAGGCGTCATGTT
4,11.0,11.0,0.0,ACGATTATCCCATAAAATGTGAACGATGCCGAAAGAAATAAAATTAGTTACTGCATTTGACGTTTGGATGTAAGTTTATCATTCGTAATAGAAATGAGGGGTGGTATGTTGGTACCCAATTAAAAAAGAACGCCTTATTTATTGATGATTGTTCCCCCGGAGTATGCCGTCCGAATAAGG
5,2.0,2.0,0.0,ACGATTTACCCGCAAAACGGGAGCGACGCCGCAAGAAACAAAATTAGTTCTTGTATTTGACATTTGGATGAAAGATTATCATTTGAAATACAATTGTGTTGAGGTATGTTGCTAGCCAATTAAAAAAGAACGCCATATTTATTGATGATTGATCCCCCGGACATCCCCCTCCTTCTAATC


### Load in sequences and transform to array

Import some helper functions. In the future this will be a package and these functions will be imported.

In [23]:
include("../src/seq_utils.jl")

transform_seq

We start by transforming the letter sequences into integers to use them as indices when accessing energy matrices. In the following matrix, each row resembles a sequence.

In [24]:
s = transform_seq.(df.seq)
VA = VectorOfArray(s)
arr = transpose(convert(Array,VA))[:, 1:160]
#JLD.save("data.jld", "data", arr_real)
#JLD.load("data.jld")["data"]

913×160 Array{Int64,2}:
 1  2  1  1  4  4  4  2  1  2  2  1  4  …  4  4  3  1  2  1  2  2  3  2  3  3
 1  2  3  1  3  4  4  2  2  2  2  1  4     4  4  3  1  4  2  2  2  2  2  3  3
 1  2  3  1  3  4  4  2  2  2  2  1  4     4  4  3  1  4  2  2  2  2  2  3  3
 1  2  3  1  4  4  1  4  2  2  2  1  4     4  4  3  4  4  2  2  2  2  2  3  3
 1  2  3  1  4  4  4  1  2  2  2  3  2     4  4  3  1  4  2  2  2  2  2  3  3
 1  2  3  1  4  4  4  1  2  2  2  3  2  …  4  4  3  1  4  2  2  2  2  2  3  3
 1  2  3  1  4  4  4  1  2  2  2  3  2     4  4  3  1  4  2  2  2  2  2  3  3
 1  2  3  1  4  4  4  2  2  1  2  1  4     4  4  3  3  2  2  2  2  2  2  3  3
 1  2  3  1  4  4  4  2  2  2  2  1  4     4  1  3  1  4  2  2  2  2  2  3  3
 1  2  3  1  4  4  4  2  2  2  2  1  4     4  4  3  1  4  2  2  2  2  2  3  3
 1  2  3  1  4  4  4  2  2  2  2  1  4  …  4  4  3  1  4  2  2  2  2  2  3  3
 1  2  3  1  4  4  4  2  2  2  2  1  4     4  4  3  1  4  2  2  2  2  2  2  3
 1  2  3  1  4  4  4  2  2  2  2  1  4  

What I learned from studying the mpathic package is that they transform this matrix into a sparse binary matrix. Here in Julia we will keep it as a dense matrix to give it to stan later.

In [25]:
function make_binary(arr::Array{Int64,2}, n::Int64)
    nrows = size(arr, 1)
    ncols = size(arr, 2)
    bin_arr = zeros(Float64, nrows, n * ncols)
    for i in 1:nrows
        for j in 1:ncols
            bin_arr[i, n * (j - 1) + arr[i, j]] = 1
        end
    end
    return bin_arr
end

bin_arr = make_binary(arr, 4)

913×640 Array{Float64,2}:
 1.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  …  0.0  1.0  0.0  0.0  0.0  1.0  0.0
 1.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0     0.0  1.0  0.0  0.0  0.0  1.0  0.0
 1.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0     0.0  1.0  0.0  0.0  0.0  1.0  0.0
 1.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0     0.0  1.0  0.0  0.0  0.0  1.0  0.0
 1.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0     0.0  1.0  0.0  0.0  0.0  1.0  0.0
 1.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  …  0.0  1.0  0.0  0.0  0.0  1.0  0.0
 1.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0     0.0  1.0  0.0  0.0  0.0  1.0  0.0
 1.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0     0.0  1.0  0.0  0.0  0.0  1.0  0.0
 1.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0     0.0  1.0  0.0  0.0  0.0  1.0  0.0
 1.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0     0.0  1.0  0.0  0.0  0.0  1.0  0.0
 1.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  …  0.0  1.0  0.0  0.0  0.0  1.0  0.0
 1.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0     1.0  0.0  0.0  0.0  0.0  1.0  0.0
 1.0  0.0  0.0  0.0  0.0  1.0  0.0  0.

In [26]:
function find_wt(bin_arr, ct_df, l::Int, n::Int)
    freqs_arr = zeros(Float64, l * n)
    for i in 1:size(bin_arr, 1)
        inds = findall(x -> x==1., bin_arr[i, :])
        freqs_arr[inds] .+= 1
    end
    r_freqs_arr = reshape(freqs_arr, n, l)
    maxima = argmax(r_freqs_arr, dims=1)
    wt_arr = zeros(Float64, l * n)
    
    for j in 1:l
        wt_arr[n * (j - 1) + maxima[j][1]] = 1
    end
    return wt_arr
end

find_wt(bin_arr, df[!, :ct], 160, 4)

640-element Array{Float64,1}:
 0.0
 0.0
 0.0
 1.0
 0.0
 1.0
 0.0
 0.0
 0.0
 0.0
 1.0
 0.0
 1.0
 ⋮
 0.0
 1.0
 0.0
 0.0
 0.0
 0.0
 1.0
 0.0
 0.0
 0.0
 1.0
 0.0

What we are looking to infer is an energy matrix, which takes a sequence as input at outputs the binding energy at each position independently. The binding energy of each sequence is obtained by independently summing up the contributions at each position.



The first step in a bayesian workflow is to define a prior for the parameters in the energry matrix. Both negative and positive contributions are possible. We start of by choosing a normal prior. There is no absolute scale for binding energies here, so we might have to gauge the priors later to obtain a converging inference.

In [27]:
theta = range(-0.01, stop=0.01, length=100)
f_theta = pdf.(Normal(0, 0.001), theta)
plot(theta, f_theta, line_width=2)

<!DOCTYPE html>
 
 
 Plots.jl

Once we chose appropriate priors, we have to define a likelihood function. From the paper we know that the liklihood is given by

\begin{align}
    L(D|\theta) \propto 2^{N I(D|\theta)},
\end{align}

where $I(E,\mu)$ is the mutual information between binding energy and expression. The goal of bayesian inference is to maximize the posterior distribution $\pi(\theta|D)$, which is given by Bayes theorem as the product of the likelihood and the priors,

\begin{align}
    \pi(\theta|D) \propto L(D|\theta)\,\pi(\theta).
\end{align}

In general not the posterior is maximized, but the log of the posterior, which will retrieve the same result due to the monotonic behavior of the log. So what we will actually be maximizing will be 

\begin{align}
    \log\pi(\theta|D) \propto \log L(D|\theta)+\log\pi(\theta).
\end{align}

The mutual information is given by


\begin{align}
    I(D|\theta) = \sum_{\mu}\sum_{E}p(E, \mu)\log_2 \left( \frac{p(E, \mu)}{p(E)p(\mu)} \right),
\end{align}

which is the mutual information between binding energies and observing gene expression.

To compute the mutual information, we need to obtain the joint probability $p(E,\mu)$. Once we have that in hand, we can obtain the marginal distributions by computing $p(E) = \sum_\mu p(E,\mu)$ and $p(\mu) = \sum_E p(E,\mu)$. The data we have as input contains sequences, which can be mapped to an energy for given parameters $\theta$, as well as counts in DNA and mRNA sequencing data. So let's draw parameters from the priors and compute energies for all the sequences.

In [28]:
# Create a random energy matrix and gauge
n_seqs = size(arr, 1)
theta = rand(Normal(0, 1), 4*160);

In [29]:
M = [3/4 -1/4 -1/4 -1/4; 
     -1/4 3/4 -1/4 -1/4; 
     -1/4 -1/4 3/4 -1/4; 
     -1/4 -1/4 -1/4 3/4]
B = zeros(Float64, 4*160, 4*160)
for i in 1:160
    B[4*(i-1)+1:4*(i-1)+4, 4*(i-1)+1:4*(i-1)+4] = M
end
theta = B * theta;

In [30]:
theta = theta / sqrt(theta' * theta / 3) * sqrt(160)
println(sum(var(reshape(theta, 4, 160), dims=1)))

160.00000000000003


In [31]:
# Compute energies
E_measured = zeros(Float64, n_seqs)
E_measured = bin_arr * theta;

Let's try to compute the distributions of binding energies.

In [32]:
p_E = zeros(Float64, 2, n_seqs)
total_counts = sum(df.ct)

for (i, counts) in enumerate([:ct_0, :ct_1])
    for j in 1:n_seqs
        p_E[i, j] = df[j, counts] / total_counts
    end
end

plot(E_measured, p_E[1, :], label="p(0, E)", xlabel="E", markersize=2, seriestype=:scatter)
plot!(E_measured, p_E[2, :], label="p(1, E)", seriestype=:scatter, markersize=2)

<!DOCTYPE html>
 
 
 Plots.jl

In [42]:
df[!, :E] = E_measured
cut_df = df[!, [:ct, :ct_0, :ct_1, :E]];
cut_df[!, :E] = round.(cut_df.E, digits=2)
gd = groupby(cut_df, :E)
sparse_df = sort!(combine(gd, [:ct, :ct_0, :ct_1] .=> sum))

,E,ct_sum,ct_0_sum,ct_1_sum
,Float64,Float64,Float64,Float64
1,-17.48,1.0,0.0,1.0
2,-17.12,29.0,10.0,19.0
3,-16.85,11.0,10.0,1.0
4,-16.72,13.0,12.0,1.0
5,-15.26,9.0,9.0,0.0
6,-13.22,1.0,1.0,0.0
7,-12.96,6.0,3.0,3.0
8,-12.82,12.0,8.0,4.0
9,-12.42,9.0,4.0,5.0


We see that the distributions seem to be very noisy. To obtain a more smooth distribution, we are going to perform a kernel density estimation. 

In [16]:
E_arr = range(minimum(E_measured), stop=maximum(E_measured), length=500)
h = 2

KDE_0 = zeros(Float64, 500)
KDE_1 = zeros(Float64, 500)
ct0 = df[!, :ct_0]
ct1 = df[!, :ct_1]

for i in eachindex(E_arr)
    for j in eachindex(E_measured)
        KDE_0[i] += ct0[j] * 1/(h*total_counts) * exp(-1/2h^2 * (E_arr[i] - E_measured[j])^2)
        KDE_1[i] += ct1[j] * 1/(h*total_counts) * exp(-1/2h^2 * (E_arr[i] - E_measured[j])^2)
    end
end
Z = (sum(KDE_0) + sum(KDE_1))
KDE_0 /= Z
KDE_1 /= Z

plot(E_arr, KDE_0, label="p_KDE(0, E)", xlabel="E", linewidth=2)
plot!(E_arr, KDE_1, label="p_KDE(1, E)", linewidth=2)

<!DOCTYPE html>
 
 
 Plots.jl

In [252]:
I = 0
Z = sum(KDE_0) + sum(KDE_1)
KDE_0 /= Z
KDE_1 /= Z
pmu0 = sum(KDE_0)
pmu1 = sum(KDE_1)
for i in 1:500
    I += KDE_0[i] * log2(KDE_0[i]/((KDE_1[i] + KDE_0[i]) * pmu0))
    I += KDE_1[i] * log2(KDE_1[i]/((KDE_1[i] + KDE_0[i]) * pmu1))
end
I

0.010495409667298818

In [253]:
stanfile_test = open("../stan_files/model_test.stan") do file
    read(file, String)
end
stanmodel_test = Stanmodel(
    model=stanfile_test,
    name="model_test",
    nchains=1,
    thin=1,
    num_warmup=100,
    num_samples=100,
    Sample(algorithm=CmdStan.Fixed_param())
)


File /Users/tomroschinger/git/jregseq/tom_notebooks/tmp/model_test.stan will be updated.



  name =                    "model_test"
  nchains =                 1
  num_samples =             100
  num_warmup =                100
  thin =                    1
  monitors =                String[]
  model_file =              "model_test.stan"
  data_file =               ""
  output =                  Output()
    file =                    ""
    diagnostics_file =        ""
    refresh =                 100
  pdir =                   "/Users/tomroschinger/git/jregseq/tom_notebooks"
  tmpdir =                 "/Users/tomroschinger/git/jregseq/tom_notebooks/tmp"
  output_format =           :array
  method =                  Sample()
    num_samples =             100
    num_warmup =              100
    save_warmup =             false
    thin =                    1
    algorithm =               Fixed_param()
    adapt =                   Adapt()
      gamma =                   0.05
      delta =                   0.8
      kappa =                   0.75
      t0 =                

In [254]:
data = Dict(
    "L_S" => 160, 
    "N_S" => nrow(df), 
    "seqs" => bin_arr, 
    "ct_0"=>convert(Array{Int64}, df.ct_0),
    "ct_1"=>convert(Array{Int64}, df.ct_1),
    "n"=>sum(convert(Array{Int64}, df.ct)),
    "centering_matrix"=>B,
)

a, chains, b = stan(stanmodel_test, data, summary=false);

Extract the parameters.

In [255]:
d = collect_params_from_chain(b, chains[:, :, 1])

Dict{Any,Any} with 15 entries:
  "E_i"           => [23.3157; 18.7153; … ; 2.15887; 32.0503]
  "accept_stat__" => [0.0; 0.0; … ; 0.0; 0.0]
  "E_min"         => [-6.70072; -8.53556; … ; -29.6813; -3.27947]
  "p_mu1"         => [6.84513; 7.53985; … ; 6.45304; 5.81504]
  "L"             => [715.143; 757.002; … ; 771.799; 812.827]
  "E_max"         => [23.3157; 18.7153; … ; 2.15887; 32.0503]
  "theta"         => [0.0837646 0.829729 … 0.320402 -0.360776; -0.624531 0.6290…
  "d_E"           => [0.0601532; 0.0546108; … ; 0.063808; 0.0708012]
  "p_E_1"         => [6.01575e-11 6.02991e-11 … 6.01575e-11 6.01575e-11; 5.4652…
  "p_mu0"         => [9.7779; 10.7574; … ; 9.21528; 8.30451]
  "p_E_0"         => [0.000144854 4.67574e-5 … 2.33787e-5 7.24269e-5; 0.0003947…
  "norm"          => [16.623; 18.2973; … ; 15.6683; 14.1195]
  "lp__"          => [0.0; 0.0; … ; 0.0; 0.0]
  "E_arr"         => [4.59649 9.0655 … 6.61315 8.53355; 2.94709 10.28 … 9.19699…
  "Info"          => [0.124545; 0.131835; … ; 0.

In [70]:
E_min_test = d["E_min"][1, :][1]
E_max_test = d["E_max"][1, :][1]
KDE_0_test = d["p_E_0"][1, :]
KDE_1_test = d["p_E_1"][1, :]

E_arr_test = range(E_min_test, E_max_test, length=500)

plot(E_arr_test, KDE_0_test, label="p_KDE(0, E) stan", xlabel="E", linewidth=2)
plot!(E_arr_test, KDE_1_test, label="p_KDE(1, E) stan", linewidth=2)


<!DOCTYPE html>
 
 
 Plots.jl

This looks pretty similar indeed!

## Inference

Now we are going to try and reproduce these steps on stan. Therefore we first write a file in which we only use the `generated quantities` section, and do not actually perform an inference. We do this extra step to confirm that the functions we wrote are actually computing what we think they do, and save a lot of time by not running an inference.

In [266]:
stanfile = open("../stan_files/model.stan") do file
    read(file, String)
end
stanmodel = Stanmodel(
    model=stanfile,
    name="model",
    nchains=4,
    thin=5,
    num_warmup=1000,
    num_samples=1000
)


File /Users/tomroschinger/git/jregseq/tom_notebooks/tmp/model.stan will be updated.



  name =                    "model"
  nchains =                 4
  num_samples =             1000
  num_warmup =                1000
  thin =                    5
  monitors =                String[]
  model_file =              "model.stan"
  data_file =               ""
  output =                  Output()
    file =                    ""
    diagnostics_file =        ""
    refresh =                 100
  pdir =                   "/Users/tomroschinger/git/jregseq/tom_notebooks"
  tmpdir =                 "/Users/tomroschinger/git/jregseq/tom_notebooks/tmp"
  output_format =           :array
  method =                  Sample()
    num_samples =             1000
    num_warmup =              1000
    save_warmup =             false
    thin =                    5
    algorithm =               HMC()
      engine =                  NUTS()
        max_depth =               10
      metric =                  CmdStan.diag_e
      stepsize =                1.0
      stepsize_jitter =      

In [268]:
data = Dict(
    "L_S" => 160, 
    "N_S" => nrow(df), 
    "seqs" => bin_arr, 
    "ct_0"=>convert(Array{Int64}, df.ct_0),
    "ct_1"=>convert(Array{Int64}, df.ct_1),
    "n"=>sum(convert(Array{Int64}, df.ct)),
    "centering_matrix"=>B
)

a, chains, b = stan(stanmodel, data, summary=false)
JLD.save("chains.jld", "chains", chains)
JLD.save("chains.jld", "b", b)




An error occurred while running the previously compiled Stan program.

Please check the contents of file model_run.log and the'command' field in the Stanmodel, e.g. stanmodel.command.



ErrorException: Return code = -5

In [44]:
chains, b = CmdStan.read_samples(stanmodel)

([629.591 0.981791 … 0.00764534 0.00781723; 635.494 0.935513 … -0.0129441 -0.000229388; … ; 656.583 0.881208 … -0.0102859 0.000715627; 642.369 0.986718 … -0.00413148 0.00374771]

[495.937 0.994909 … 0.018277 9.51355e-5; 510.037 0.635905 … 0.00940894 -0.000816384; … ; 552.874 0.997412 … 0.0161236 0.0029821; 512.243 0.788358 … 0.0164292 -0.0112969]

[553.46 0.806247 … -0.00581208 -0.0245972; 597.132 0.817921 … 0.000944044 -0.0279758; … ; 568.531 0.93386 … 0.00603253 -0.0226549; 558.514 0.586167 … -0.00410848 -0.0275406]

[642.893 0.796672 … 0.0010485 0.0145532; 658.52 0.580392 … 0.00259001 -0.00367083; … ; 640.719 0.9565 … -0.00320268 -0.000927209; 632.937 0.904375 … 0.00329205 -0.00851132], ["lp__", "accept_stat__", "stepsize__", "treedepth__", "n_leapfrog__", "divergent__", "energy__", "theta.1", "theta.2", "theta.3"  …  "theta.631", "theta.632", "theta.633", "theta.634", "theta.635", "theta.636", "theta.637", "theta.638", "theta.639", "theta.640"])

In [45]:
p=corner(b, chains, parameters=[("theta", 1), ("theta", 2), ("theta", 3)], color_by_chain=true, plot_ecdf=true)

<!DOCTYPE html>
 
 
 Plots.jl

Let's compute the mutual information. Therefore we renormalize the distributions.

In [46]:
trace_plot(b, chains, [("theta", 1), ("theta", 2),("theta", 3)])

<!DOCTYPE html>
 
 
 Plots.jl

Let's look at the interpolated probability distributions and see if they look similar. They won't look exactly the same, since the parameters $\theta$ are generated randomly.

In [56]:
check_all_diagnostics(b,chains)

Rhat for parameter theta.1 is 1.3872678197531352.
Rhat for parameter theta.2 is 1.2815833235697491.
Rhat for parameter theta.3 is 1.3085961165525382.
Rhat for parameter theta.4 is 1.4545204941091818.
Rhat for parameter theta.5 is 1.1375942859488397.
Rhat for parameter theta.6 is 1.7259472599052619.
Rhat for parameter theta.7 is 1.4973050309559903.
Rhat for parameter theta.8 is 1.3586239962800557.
Rhat for parameter theta.9 is 1.1003094606559534.
Rhat for parameter theta.10 is 1.102215748695185.
Rhat for parameter theta.11 is 1.6500915736142627.
Rhat for parameter theta.12 is 1.9713941699321895.
Rhat for parameter theta.13 is 1.5634607103271667.
Rhat for parameter theta.14 is 1.6023945704075713.
Rhat for parameter theta.15 is 1.5503577977489567.
Rhat for parameter theta.16 is 1.5495487028439268.
Rhat for parameter theta.17 is 1.0369145173136405.
Rhat for parameter theta.18 is 1.3143254527391326.
Rhat for parameter theta.19 is 1.4133639931699926.
Rhat for parameter theta.20 is 1.49991273